In [ ]:
! pip install datasets transformers rouge-score nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns

import matplotlib.pyplot as plt
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

4.19.2


# Fine-tuning a model on a summarization task

In [ ]:
model_checkpoint = "t5-small"

## Loading the dataset

In [ ]:
from datasets import load_dataset, load_metric

# raw_datasets = load_dataset("xsum")
metric = load_metric("rouge")

In [ ]:
print( os.listdir('/content/drive') )
data = pd.read_json("/content/drive/MyDrive/datasets_qa/qa_df.json")
data.head(n=10)

['.file-revisions-by-id', '.shortcut-targets-by-id', 'MyDrive', 'Othercomputers', '.Trash-0']


,book_title,chapter_name,context,answer_start,text
0,Little Women,Little_Women_split_001.html,table of contents from the pages of little wom...,"[239, 159]","[burdens, pilgrims]"
1,Little Women,Little_Women_split_002.html,"from the pages of little women "" christmas won...","[1848, 624, 1496]",[it takes people a long time to learn the diff...
2,Little Women,Little_Women_split_002.html,"mysterious and important. meg observed it, but...","[246, 925]",[it takes people a long time to learn the diff...
3,Little Women,Little_Women_split_004.html,published by barnes & noble books 122 fifth av...,[],[]
4,Little Women,Little_Women_split_006.html,louisa may alcott louisa may alcott was born o...,[],[]
5,Little Women,Little_Women_split_006.html,"in 1858 when her younger sister, elizabeth, di...",[],[]
6,Little Women,Little_Women_split_007.html,the world of louisa may alcott and little wome...,[],[]
7,Little Women,Little_Women_split_007.html,1844. 1844 emerson publishes essays : second s...,[],[]
8,Little Women,Little_Women_split_007.html,she attends lectures by the liberal clergyman ...,[],[]
9,Little Women,Little_Women_split_007.html,book form later in the year to great praise. a...,[],[]


In [ ]:
data.iloc[0]["text"]

['burdens', 'pilgrims']

In [ ]:
sum_df = pd.DataFrame(columns=["text","id","quote"])

quote_count =0
i=0
for idx in range(len(data)):
  for quote in data.iloc[idx]["text"]:
    sum_df.loc[i]=[data.iloc[idx]["context"],i+1,quote]
    i+=1
    quote_count+=1




In [ ]:
sum_df.shape

(4705, 3)

In [ ]:
quote_count

4705

In [ ]:
sum_df.columns

Index(['text', 'id', 'quote'], dtype='object')

In [ ]:
from datasets import Dataset
from datasets.dataset_dict import DatasetDict

dataset = Dataset.from_pandas(sum_df)
train_testvalid = dataset.train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
train_test_valid_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']
    })


In [ ]:
sum_df[:5]

,text,id,quote
0,table of contents from the pages of little wom...,1,burdens
1,table of contents from the pages of little wom...,2,pilgrims
2,"from the pages of little women "" christmas won...",3,it takes people a long time to learn the diffe...
3,"from the pages of little women "" christmas won...",4,"have regular hours for work and play, make eac..."
4,"from the pages of little women "" christmas won...",5,"jo's face was a study next day, for the secret..."


In [ ]:
raw_datasets = train_test_valid_dataset

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'id', 'quote', '__index_level_0__'],
        num_rows: 3764
    })
    test: Dataset({
        features: ['text', 'id', 'quote', '__index_level_0__'],
        num_rows: 471
    })
    valid: Dataset({
        features: ['text', 'id', 'quote', '__index_level_0__'],
        num_rows: 470
    })
})

In [ ]:
raw_datasets["train"][0]

{'__index_level_0__': 658,
 'id': 659,
 'quote': 'deliquescent.',
 'text': "later on in life, you expect a bit of rest, don't you? you think you deserve it. i did, anyway. but then you begin to understand that the reward of merit is not life's business. also, when you are young, you think you can predict the likely pains and bleaknesses that age might bring. you imagine yourself being lonely, divorced, widowed ; children growing away from you, friends dying. you imagine the loss of status, the loss of desire - and desirability. you may go further and consider your own approaching death, which, despite what company you may muster, can only be faced alone. but all this is looking ahead. what you fail to do is look ahead, and then imagine yourself looking back from that future point. learning the new emotions that time brings. discovering, for example, that as the witnesses to your life diminish, there is less corroboration, and therefore less certainty, as to what you are or have been. e

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])

,text,id,quote,__index_level_0__
0,"be a wonderful side effect. but at the end of the day, i do what i do because i like doing it. i have a friend who's a nun who has spent her entire life working to help the homeless of philadelphia. she is something close to a living saint. she is a tireless advocate for the poor and the suffering and the lost and the abandoned. and do you know why her charitable outreach is so effective? because she likes doing it. because it's enjoyable for her. otherwise it wouldn't work. otherwise, it would just be hard duty and grim martyrdom. but sister mary scullion is no martyr. she's a cheerful soul who's having a wonderful time living out the existence that best suits her nature and most brings her to life. it just so happens that she takes care of a lot of other people in the process - - but everyone can sense her genuine enjoyment behind the mission, which is ultimately why her presence is so healing. it's okay if your work is fun for you, is what i'm saying. it's also okay if your work is healing for you, or fascinating for you, or redemptive for you, or if it's maybe just a hobby that keeps you from going crazy. it's even okay if your work is totally frivolous. that's allowed. it's all allowed. your own reasons to create are reason enough. merely by pursuing what you love, you may inadvertently end up helping us plenty. ( "" there is no love which does not become help, "" taught the theologian paul tillich. ) do whatever brings you to life, then. follow your own fascinations, obsessions, and compulsions. trust them. create whatever causes a revolution in your heart. the rest of it will take care of itself. schooling i never got an advanced degree in writing. i don't have an advanced degree in anything, actually. i graduated from nyu with a bachelor's degree in political science ( because you have to major in something ) and i still feel lucky to have received what i consider to have been an excellent, old - fashioned, broad - minded liberal arts education. while i always knew that i wanted to be a writer, and while i took a few writing classes as an undergrad, i chose not to seek out a master's of fine arts in creative writing once i was finished at nyu. i was suspicious",540,"do whatever brings you to life, then.",539
1,". doc daneeka hated to fly. he felt imprisoned in an airplane. in an airplane there was absolutely no place in the world to go except to another part of the airplane. doc daneeka had been told that people who enjoyed climbing into an airplane were really giving vent to a subconscious desire to climb back into the womb. he had been told this by yossarian, who made it possible for dan daneeka to collect his flight pay each month without ever climbing back into the womb. yossarian would persuade mcwatt to enter doc daneeka's name on his flight log for training missions or trips to rome. "" you know how it is, "" doc daneeka had wheedled, with a sly, conspiratorial wink. "" why take chances when i don't have to? "" "" sure, "" yossarian agreed. "" what difference does it make to anyone if i'm in the plane or not? "" "" no difference. "" "" sure, that's what i mean, "" doc daneeka said. "" a little grease is what makes this world go round. one hand washes the other. know what i mean? you scratch my back, i'll scratch yours. "" yossarian knew what he meant. "" that's not what i meant, "" doc daneeka said, as yossarian began scratching his back. "" i'm talking about co - operation. favors. you do a favor for me, i'll do one for you. get it? "" "" do one for me, "" yossarian requested. "" not a chance, "" doc daneeka answered. there was something fearful and minute about doc daneeka as he sat despondently outside his tent in the sunlight as often as he could, dressed in khaki summer trousers and a short - sleeved summer shirt that was bleached almost to an antiseptic gray by the daily laundering to which he had it subjected. he was like a man who had grown frozen with horror once and 

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [ ]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings:

In [ ]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

## Preprocessing the data

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:161: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


If you are using one of the five T5 checkpoints we have to prefix the inputs with "summarize:" (the model can also translate and it needs the prefix to know which task it has to perform).

In [ ]:
# if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
#     prefix = "summarize: "
# else:
#     prefix = ""

In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["quote"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[865, 30, 16, 280, 6, 25, 1672, 3, 9, 720, 13, 880, 6, 278, 31, 17, 25, 58, 25, 317, 25, 9092, 34, 5, 3, 23, 410, 6, 6161, 5, 68, 258, 25, 1731, 12, 734, 24, 8, 9676, 13, 7565, 19, 59, 280, 31, 7, 268, 5, 92, 6, 116, 25, 33, 1021, 6, 25, 317, 25, 54, 9689, 8, 952, 1406, 7, 11, 3, 2296, 1639, 655, 15, 7, 24, 1246, 429, 830, 5, 25, 3034, 909, 271, 23633, 6, 7759, 26, 6, 23428, 15, 26, 3, 117, 502, 1710, 550, 45, 25, 6, 803, 13677, 5, 25, 3034, 8, 1453, 13, 2637, 6, 8, 1453, 13, 3667, 3, 18, 11, 93, 23, 52, 2020, 5, 25, 164, 281, 856, 11, 1099, 39, 293, 16009, 1687, 6, 84, 6, 3, 3565, 125, 349, 25, 164, 398, 49, 6, 54, 163, 36, 7865, 2238, 5, 68, 66, 48, 19, 479, 2177, 5, 125, 25, 5124, 12, 103, 19, 320, 2177, 6, 11, 258, 3034, 909, 479, 223, 45, 24, 647, 500, 5, 1036, 8, 126, 7848, 24, 97, 3200, 5, 17452, 6, 21, 677, 6, 24, 38, 8, 20837, 12, 39, 280, 26999, 6, 132, 19, 705, 4301, 23801, 2661, 6, 11, 2459, 705, 24848, 6, 38, 12, 125, 25, 33, 42, 43, 118, 5, 237, 3, 99, 25, 

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-quote-sum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
# data_collator

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp half precision backend


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__, text, quote, id. If __index_level_0__, text, quote, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3764
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 708


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.601559,14.759900,6.218500,13.237100,13.721200,18.521300
2,No log,0.563475,16.821400,8.358800,15.049900,15.571000,18.508500
3,0.794300,0.555365,16.626100,8.036300,14.852100,15.338400,18.593600


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__, text, quote, id. If __index_level_0__, text, quote, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 470
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__, text, quote, id. If __index_level_0__, text, quote, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 470
  Batch size = 16
Saving model checkpoint to t5-small-finetuned-quote-sum/checkpoint-500
Configuration saved in t5-small-finetuned-quote-sum/checkpoint-500/config.json
Model weights saved in t5-small-finetuned-quote-sum/checkpoint-500/pytorch_model.bin
t

TrainOutput(global_step=708, training_loss=0.7647534171066716, metrics={'train_runtime': 324.5433, 'train_samples_per_second': 34.794, 'train_steps_per_second': 2.182, 'total_flos': 1528279622221824.0, 'train_loss': 0.7647534171066716, 'epoch': 3.0})

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.save_model("/content/drive/MyDrive/datasets_qa/test-sum-trained")


Saving model checkpoint to /content/drive/MyDrive/datasets_qa/test-sum-trained
Configuration saved in /content/drive/MyDrive/datasets_qa/test-sum-trained/config.json
Model weights saved in /content/drive/MyDrive/datasets_qa/test-sum-trained/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/datasets_qa/test-sum-trained/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/datasets_qa/test-sum-trained/special_tokens_map.json


In [ ]:
set_type = "valid"

In [ ]:
raw_predictions = trainer.predict(tokenized_datasets[set_type])


The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__, text, quote, id. If __index_level_0__, text, quote, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 470
  Batch size = 16


In [ ]:
# tokenizer.batch_decode(raw_predictions.predictions[0])
raw_predictions

PredictionOutput(predictions=array([[   0,    3,    9, ...,   11,    3,    7],
       [   0,    3,   88, ...,    0,    0,    0],
       [   0,    8, 1997, ...,   18, 8114,    3],
       ...,
       [   0, 1028,   15, ...,   23, 2124,    6],
       [   0,   21, 4040, ..., 1162, 5963,  718],
       [   0, 1911, 6626, ...,    9, 6377,    6]]), label_ids=array([[  163,     8, 31033, ...,  -100,  -100,  -100],
       [  227,  2361, 12010, ...,   160,   323,     1],
       [    8,  2195,   141, ...,  -100,  -100,  -100],
       ...,
       [   25,  3223,     5, ...,  -100,  -100,  -100],
       [   16,   685,     6, ...,  -100,  -100,  -100],
       [    3,    88,     3, ...,  -100,  -100,  -100]]), metrics={'test_loss': 0.555364727973938, 'test_rouge1': 16.6261, 'test_rouge2': 8.0363, 'test_rougeL': 14.8521, 'test_rougeLsum': 15.3384, 'test_gen_len': 18.5936, 'test_runtime': 14.9757, 'test_samples_per_second': 31.384, 'test_steps_per_second': 2.003})

In [ ]:
tokenized_datasets[set_type]

Dataset({
    features: ['text', 'id', 'quote', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 470
})

In [ ]:
tokenized_datasets[set_type]["labels"][0]

[163,
 8,
 31033,
 106,
 343,
 1944,
 30,
 5,
 24,
 31,
 7,
 31033,
 106,
 343,
 7,
 21,
 25,
 5,
 1]

In [ ]:
word_predictions = tokenizer.batch_decode(raw_predictions.predictions)

In [ ]:
pd.DataFrame({"text":tokenized_datasets[set_type]["text"],"quote":tokenized_datasets[set_type]["quote"],"predicted":word_predictions})

,text,quote,predicted
0,". perfect, i thought. camouflage, you see. a h...",only the trombonist played on. that's tromboni...,<pad> a beehive of consternation in all major ...
1,"to get her up. tommy, hurry! he lifted the boo...","after perhaps thirty meters, just as a soldier...",<pad> he collected her punches as if they were...
2,"with them, for as long as it takes. and watch....","the tree had been dead a long time, but it see...","<pad> the sun was long gone, and only the cold..."
3,"eyes and blood on his hand, clutching at the b...","unbearable claustrophobia of the soul,","<pad> it was nothing i hadn't thought of, plen..."
4,", and before very long only jon and ghost rema...","strike hard and true, crow, or i'll come back ...",<pad> he touched the edge of the blade to mark...
...,...,...,...
465,"problem?'' i need someone to go with me, to be...",fate's way of beating us in a fair fight is to...,<pad> fate's way of beating us in a fair fight...
466,in the terms which one gentleman would use to ...,i make so many beginnings there never will be ...,<pad> jo dropped a kiss on the top of mr. laur...
467,"had he not spoke back, spoke up, would he stil...",you exist. you matter. you have value. you hav...,<pad> disembodiment. the dragon that compelled...
468,"evolution been more advanced, we would probabl...","in fact, overwhelmingly museum displays are ar...","<pad> for millions of years, a gigantic, fligh..."


In [ ]:
rouge_score = load_metric("rouge")
scores = rouge_score.compute(
    predictions=raw_predictions[1], references=tokenized_datasets[set_type]["labels"]
)
scores

{'rouge1': AggregateScore(low=Score(precision=0.2194975897606383, recall=1.0, fmeasure=0.3411336424648481), mid=Score(precision=0.23323636968085107, recall=1.0, fmeasure=0.35686150661975424), high=Score(precision=0.24769281914893615, recall=1.0, fmeasure=0.3716224254266193)),
 'rouge2': AggregateScore(low=Score(precision=0.21479309767130222, recall=1.0, fmeasure=0.33396342043889643), mid=Score(precision=0.22703132853074254, recall=1.0, fmeasure=0.3479762875993221), high=Score(precision=0.2418009716870499, recall=1.0, fmeasure=0.36385225861837794)),
 'rougeL': AggregateScore(low=Score(precision=0.2197286402925532, recall=1.0, fmeasure=0.34127034224236946), mid=Score(precision=0.23289561170212764, recall=1.0, fmeasure=0.3565301871323091), high=Score(precision=0.24730759640957445, recall=1.0, fmeasure=0.37242209398552917)),
 'rougeLsum': AggregateScore(low=Score(precision=0.21588721742021277, recall=0.9879307335747025, fmeasure=0.3360895694667142), mid=Score(precision=0.22853224734042554,

In [ ]:
scores["rouge1"].mid


Score(precision=0.23323636968085107, recall=1.0, fmeasure=0.35686150661975424)

In [ ]:
# trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```